In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
# Add parent dir to path, so that python finds the lenskit package
sys.path.insert(0,parentdir)

In [2]:
from lenskit.metrics import dataGenerator
from lenskit import batch, topn, util, topnFair
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, funksvd, user_knn as uknn, item_knn as iknn 
from lenskit import topn, topnFair

In [3]:
import numpy as np
import pandas as pd
import math
import pickle
%matplotlib inline

In [4]:
dataset = os.path.join(parentdir,'book_data/ratings.csv')     
ratings = pd.read_csv(dataset, sep=',',
                      names=['user', 'item', 'rating'], header=0)

data = os.path.join(parentdir,'book_data/book_author_gender.csv')

items = pd.read_csv(data, sep=',',
                      names=['item','gender'], header=0)

In [5]:
ratings.head()


,user,item,rating
0,156050,102337377,8.0
1,102032,107971879,5.0
2,14422,104568501,9.0
3,122037,101204681,8.0
4,109864,919256170,7.0


In [6]:
items = items.loc[(items["gender"] == "male") | (items["gender"] == "female")]
items = items.reset_index(drop="true")
print(items.shape)
items.head()

(4251558, 2)


,item,gender
0,113484902,male
1,100940351,male
2,104633344,female
3,202193849,male
4,101354697,male


In [7]:
ratings_joined =  pd.merge(ratings, items.set_index('item'),how="inner", on='item')

In [8]:
ratings_joined.shape

(319988, 4)

In [9]:
#rateditems = ratings.item.unique()
#rateditems = pd.DataFrame(rateditems, columns =['item'])
#rateditems = pd.merge(items.set_index('item'), on='item', how="inner"  )
#rateditems = pd.merge(rateditems, items.set_index('item'),how="inner", on='item')
#print(rateditems.shape)
#rateditems = rateditems.dropna()
#print(rateditems.shape)
#rateditems.head()




In [46]:
def create_sample( data, user_cut, item_cut):

    temp_user = data.groupby('user').item.count().to_frame('count').reset_index()
    temp_user = temp_user.sort_values("count", ascending=False)
    #temp.columns = [1,2]
    temp_user = temp_user[temp_user["count"] >= user_cut]
    #print(temp.head())
    print(temp_user.shape)
    #df.loc[df['column_name'].isin(some_values)]
    temp_user = data.loc[data["user"].isin(temp_user.user)]
    
    temp_item = data.groupby('item').user.count().to_frame('count').reset_index()
    temp_item = temp_item.sort_values("count", ascending=False)
    #temp.columns = [1,2]
    temp_item = temp_item[temp_item["count"] >= item_cut]
    #print(temp.head())
    print(temp_user.shape)
    #df.loc[df['column_name'].isin(some_values)]
    
    temp_item = data.loc[data["item"].isin(temp_item.item)]
    
    toreturn = pd.merge(temp_user, temp_item, on="item",  how="inner")

return(toreturn)
#ratings.groupby('user').item.count()

In [88]:
res = create_sample(ratings_joined, 20, 30)
res.item.nunique()

(2746, 2)
(165889, 4)


1296

In [52]:
res.item.nunique()

1296

In [89]:
def create_sample_user( data, user_cut, item_frac):

    temp_user = data.groupby('user').item.count().to_frame('count').reset_index()
    temp_user = temp_user.sort_values("count", ascending=False)
    #temp.columns = [1,2]
    temp_user = temp_user[temp_user["count"] >= user_cut]
    #print(temp.head())
    print(temp_user.shape)
    #df.loc[df['column_name'].isin(some_values)]
    temp_user = data.loc[data["user"].isin(temp_user.user)]
   
    temp_item = data.item.sample(frac=item_frac)
    print(temp_item.shape) 
    temp_item = data.loc[data["item"].isin(temp_item)]
    
    toreturn = pd.merge(temp_user, temp_item, on="item",  how="inner")

    return(toreturn)




In [91]:
res = create_sample_user(ratings_joined, 20, 0.005)
res.item.nunique()

(2746, 2)
(1600,)


1290

In [ ]:
68102